In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import math
import pylab
import scipy.stats as stats
%matplotlib inline

###Final Modeling
cols = [
 'id',
    'host_id',
    'host_listings_count',
    'zipcode',
    'neighbourhood',
    'property_type',
    'latitude',
    'longitude',
    'room_type',
    'accommodates',
    'minimum_nights',
    'bedrooms',
    'beds',
    'bed_type',
    'bathrooms',
    'cleaning_fee',
    'price',
    'review_scores_rating',
    'reviews_per_month',
    'calculated_host_listings_count',
    'host_listings_count',
    'availability_30',
    'availability_365',
    'host_since',
    'host_is_superhost',
    'maximum_nights',
    'extra_people',
    'security_deposit',
    'number_of_reviews', 
    'host_response_rate',
    'host_response_time']

df1 = pd.read_csv('C:/Users/fahmida/Desktop/Rental/airbnb/Listings/listings.csv.gz', usecols=cols)

In [3]:
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings

from collections import Counter
from scipy.stats.stats import pearsonr
from string import ascii_letters


pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')
%matplotlib inline

def correction(x):
    '''
    Columns value corrections
    '''
    if type(x)==str:
        x=x.replace('$','')
        x=x.replace(',','')
        x=float(x)    
    return (x)

def correction2(x):
    '''
    Columns value corrections
    '''
    if type(x)==str:
        x=x.replace('%','')
        x=float(x)/100.0
    return (x)

def to_int(x):
    '''
    Columns value corrections
    '''
    if x=='f':
        x=x.replace('f','0')
    elif x=='t':
        x=x.replace('t','1')
    else:
        x= '0'
    return int(x)

def changeTime(x):
    '''
    change host_response_time columns from string into numerical.
    '''
    if x == 'within an hour':
        x='1'
    elif x == 'within a few hours':
        x='4'
    elif x == 'within a day':
        x='24'
    elif x == 'a few days or more':
        x='48'
    else:
        x='96'
        
    return x


def changeStr(x):
    '''
    change back the host_response_time from the numerical into strings
    '''
    if x == 1:
        x='within an hour'
    elif x == 4:
        x='within a few hours'
    elif x == 24:
        x='within a day'
    elif x == 48:
        x= 'a few days or more'
    elif x == 96:
        x= 'Not Response'
        
    return x

def createAmenities(x):
    '''
    Convert the Amenities column into more analytical words
    '''
    val = x.replace('{','').replace('}','').replace('"','').replace(' ','_').replace(',',' ')
    val = val.split()
    return val


def rangeScore(x):
    '''
    Set the bins for the score-range.
    '''
    value = ''
    if (x>= 0 and x < 10):
        value = '0-10'
    elif (x>= 10 and x < 20):
        value = '10-20'
    elif (x>= 20 and x < 30):
        value = '20-30'
    elif (x>= 30.0 and x < 40.0):
        value = '30-40'
    elif (x>= 40 and x < 50):
        value = '40-50'
    elif (x>= 50 and x < 60):
        value = '50-60'
    elif (x>= 60 and x < 70):
        value = '60-70'        
    elif (x>= 70 and x < 80):
        value = '70-80'
    elif (x>= 80 and x < 90):
        value = '80-90'
    elif (x>= 90 and x < 100):
        value = '90-100'
    elif x>= 100:
        value = '100+'
        
    return value

In [4]:
# remove NaN values from dataframe
original = len(df1)
df1 = df1.dropna(how='any', subset=[ 'id',
    'host_id',
    'host_listings_count',
    'zipcode',
    'neighbourhood',
    'property_type',
    'latitude',
    'longitude',
    'room_type',
    'accommodates',
    'minimum_nights',
    'bedrooms',
    'beds',
    'bed_type',
    'bathrooms',
    'cleaning_fee',
    'price',
    'review_scores_rating',
    'reviews_per_month',
    'calculated_host_listings_count',
    'host_listings_count',
    'availability_30',
    'availability_365',
    'host_since',
    'host_is_superhost',
    'maximum_nights',
    'extra_people',
    'security_deposit',
    'number_of_reviews', 
    'host_response_rate',
    'host_response_time',])

# convert formatting for price
df1['price'] = (df1['price'].str.replace(r'[^-+\d.]', '').astype(float))
df1['cleaning_fee'] = (df1['cleaning_fee'].str.replace(r'[^-+\d.]', '').astype(float))
df1['security_deposit'] = (df1['security_deposit'].str.replace(r'[^-+\d.]', '').astype(float))
df1['new_score_reviews'] = df1['reviews_per_month'] * df1['review_scores_rating'] / 10
df1['new_score_reviews'].fillna(0, inplace = True)
df1['score_ranges'] = df1['new_score_reviews'].apply(rangeScore)

df1 = df1[df1['accommodates'] != 0]
df1 = df1[df1['neighbourhood'] != 0]
df1 = df1[df1['bedrooms'] != 0]
df1 = df1[df1['beds'] != 0]
df1 = df1[df1['price'] != 0.00]
df1 = df1[df1['cleaning_fee'] != 0.00]
df1 = df1[df1['security_deposit'] != 0.00]
df1.drop(['new_score_reviews'],axis=1)

# convert ZipCode
df1['zipcode'] = df1['zipcode'].str.replace(r'-\d+', '')

In [5]:
print(df1)

             id    host_id  host_since  host_response_time host_response_rate  \
5         26654     113345  2010-04-25  within a few hours                90%   
19        51616     237587  2010-09-17  within a few hours                90%   
22        64003     312769  2010-12-08      within an hour               100%   
30        96336     511506  2011-04-15  within a few hours               100%   
31        98136     518383  2011-04-18        within a day                80%   
32       104096     349172  2011-01-16      within an hour               100%   
33       104722     455397  2011-03-21        within a day                50%   
34       112842     575127  2011-05-10  a few days or more                 0%   
40       137099     149566  2010-06-22        within a day                80%   
41       138006     605569  2011-05-19      within an hour               100%   
46       153000     735795  2011-06-23      within an hour               100%   
52       182573     591649  

In [6]:
for i in df1.columns:
    
    if set(df1[i])=={'t','f'}:
        df1[i]=df1[i].apply(to_int)
        
    elif set(df1[i]) == {'t','f',np.nan}:
        df1[i]=df1[i].apply(to_int)

In [7]:
df1['extra_people'] =df1['extra_people'].map(lambda x: correction(x))
# Change the rate percentage.
df1['host_response_rate'] = df1['host_response_rate'].fillna('0%').apply(correction2)
# Change time indicators
df1['host_response_time'] = df1['host_response_time'].apply(changeTime).astype(int)

In [8]:
# read to csv
df1.to_csv('C:/Users/fahmida/Desktop/Rental/airbnb/listings_clean.csv', index=False)

In [9]:
###Price Prediction Starts
